In [ ]:
import os
os.environ['KAGGLE_USERNAME']="simransohal"
os.environ['KAGGLE_KEY']="3aa7697cd9a4ba2a3715c4a29c0f0cb6"

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
ls

asl_alphabet_test/  asl_alphabet_train/  asl-alphabet.zip  sample_data/


In [ ]:
import zipfile
with zipfile.ZipFile('asl-alphabet.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
ls

asl_alphabet_test/  asl_alphabet_train/  asl-alphabet.zip  sample_data/


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import torch.hub 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [6]:
d=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model=models.resnet18(pretrained=False)

c:\users\nisha\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\users\nisha\appdata\local\programs\python\python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
ls

 Volume in drive C is OS
 Volume Serial Number is 1050-3BDA

 Directory of C:\COMP6721\ASL_to_Text_Classification

2023-03-16  01:54 PM    <DIR>          .
2023-03-11  08:57 PM    <DIR>          ..
2023-03-11  08:57 PM             1,928 .gitignore
2023-03-16  01:50 PM    <DIR>          .ipynb_checkpoints
2023-03-16  01:52 PM            18,133 Artificial_Intelligence_resnet.ipynb
2023-03-11  08:57 PM                42 CHANGE.md
2023-03-11  08:59 PM    <DIR>          Proposal
2023-03-11  08:57 PM                51 README.md
2023-03-16  01:54 PM             9,121 ShuffleNet-Signs.ipynb
2023-03-11  08:58 PM    <DIR>          signs_train
               5 File(s)         29,275 bytes
               5 Dir(s)  20,232,339,456 bytes free


In [9]:
#Getting the number of input features in fully connected layer
#Model is to be classified in just three layers
#Created a new layer and replaced the fully connected layer with it
num_classes=3
features=model.fc.in_features
model.fc=nn.Linear(features,num_classes)

In [10]:
model=model.to(d)

In [11]:
transformations=transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])

In [12]:
data=datasets.ImageFolder(r"C:\COMP6721\ASL_to_Text_Classification\signs_train\asl_alphabet_train\asl_alphabet_train",transform=transformations)

In [13]:
optimizer=Adam(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [14]:
from torch.utils.data import DataLoader
loader_train=DataLoader(data,batch_size=32,shuffle=True )
loader_test=DataLoader(data,batch_size=32,shuffle=True)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 20
total_steps = len(loader_train)

t1 = time.time()

for epoch in range(num_epochs):
    for i, data in enumerate(loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/20], Step [50/282], Loss: 0.3418, Accuracy: 84.38%
Epoch [1/20], Step [100/282], Loss: 0.0281, Accuracy: 100.00%
Epoch [1/20], Step [150/282], Loss: 0.1405, Accuracy: 93.75%
Epoch [1/20], Step [200/282], Loss: 0.0561, Accuracy: 96.88%
Epoch [1/20], Step [250/282], Loss: 0.0234, Accuracy: 100.00%
Epoch [2/20], Step [50/282], Loss: 0.0019, Accuracy: 100.00%
Epoch [2/20], Step [100/282], Loss: 0.0012, Accuracy: 100.00%
Epoch [2/20], Step [150/282], Loss: 0.0017, Accuracy: 100.00%
Epoch [2/20], Step [200/282], Loss: 0.0011, Accuracy: 100.00%
Epoch [2/20], Step [250/282], Loss: 0.0062, Accuracy: 100.00%
Epoch [3/20], Step [50/282], Loss: 0.1053, Accuracy: 96.88%
Epoch [3/20], Step [100/282], Loss: 0.0078, Accuracy: 100.00%
Epoch [3/20], Step [150/282], Loss: 0.0073, Accuracy: 100.00%
Epoch [3/20], Step [200/282], Loss: 0.0120, Accuracy: 100.00%
Epoch [3/20], Step [250/282], Loss: 0.0007, Accuracy: 100.00%
Epoch [4/20], Step [50/282], Loss: 0.0019, Accuracy: 100.00%
E

In [1]:
model.eval()
with torch.no_grad():
  correct = 0
  total=0
  for image,label in loader_test:
    images,label=images.to(d),labels.to(d)
    result=model(images)
    _,predicted=torch.max(result.data,1)
    total=total+labels.size(0)
    correct=correct+(predicted==labels).sum().items()
    print(f'Accuracy {100 * correct / total:.2f}%')
 



NameError: name 'model' is not defined